# PHMSA Company Hierarchy Analysis - LLM Agent

Automatically identifies corporate parent-subsidiary relationships using LLM + web search.

**Features:** Multi-search strategy, Joint Venture detection, Recency validation, Graph resolution

**Runtime:** ~2-4 hours for 1000 companies

In [ ]:
# Cell 1: Install Dependencies & Initialize LLM

%pip install -U langchain-community langchain-core langchain duckduckgo-search ddgs pandas networkx
dbutils.library.restartPython()

# After restart, run this cell again to initialize
from langchain_community.chat_models import ChatDatabricks
from langchain_community.tools import DuckDuckGoSearchResults

llm = ChatDatabricks(endpoint="databricks-claude-sonnet-4-5", max_tokens=1000, temperature=0.1)
search_tool = DuckDuckGoSearchResults(num_results=5)

print("✓ LLM and search tool initialized")

In [ ]:
# Cell 2: Load Data & Initialize Components

import sys
sys.path.append('/Workspace/Repos/YOUR_USERNAME/phmsa-company-hierarchy/')  # UPDATE THIS

from phmsa_hierarchy import AgentLLMValidator, HierarchyGraphBuilder, CompanyNameNormalizer

graph_builder = HierarchyGraphBuilder()

# Load PHMSA data
companies_df = spark.read.table("gshen_catalog.enbridge_sr_workshop.annual_hazardous_liquid_2024_updated")
companies_df = companies_df.select("OPERATOR_ID", "PARTA2NAMEOFCOMP", "PARTA4STREET", "PARTA4CITY", "PARTA4STATE").distinct()

# Prepare company list for validation
all_companies = [row["PARTA2NAMEOFCOMP"] for row in companies_df.select("PARTA2NAMEOFCOMP").distinct().collect()]

print(f"✓ Loaded {companies_df.count()} unique companies")
print(f"✓ Prepared {len(all_companies)} companies for validation")

In [ ]:
# Cell 3: Define Pandas UDF for Agent-Based Validation

from pyspark.sql.functions import pandas_udf, struct, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
import pandas as pd

# Define output schema
output_schema = StructType([
    StructField("OPERATOR_ID", IntegerType(), True),
    StructField("ORIGINAL_NAME", StringType(), True),
    StructField("IMMEDIATE_PARENT", StringType(), True),
    StructField("LLM_SEARCH_PARENT", StringType(), True),  # Raw LLM result before validation
    StructField("IS_JV", BooleanType(), True),  # Joint venture flag
    StructField("JV_PARTNERS", StringType(), True),  # Comma-separated list of JV partners
    StructField("PRIMARY_OPERATOR", StringType(), True),  # Who operates the JV
    StructField("CONFIDENCE", IntegerType(), True),
    StructField("REASONING", StringType(), True),
    StructField("ACQUISITION_DATE", StringType(), True),
    StructField("RECENT_CHANGE", BooleanType(), True),
    StructField("PARTA4STREET", StringType(), True),
    StructField("PARTA4CITY", StringType(), True),
    StructField("PARTA4STATE", StringType(), True),
])

@pandas_udf(output_schema)
def find_parent_agent(data: pd.DataFrame) -> pd.DataFrame:
    from langchain_community.chat_models import ChatDatabricks
    from langchain_community.tools import DuckDuckGoSearchResults
    from datetime import datetime
    import sys
    sys.path.append('/Workspace/Repos/YOUR_USERNAME/phmsa-company-hierarchy/')
    from phmsa_hierarchy import AgentLLMValidator
    
    batch_start = datetime.now()
    batch_size = len(data)
    print(f"\n{'#'*60}\n📦 BATCH START: {batch_size} companies\n{'#'*60}")
    
    llm_local = ChatDatabricks(endpoint="databricks-claude-sonnet-4-5", max_tokens=1000, temperature=0.1)
    search_tool_local = DuckDuckGoSearchResults(num_results=5)
    validator = AgentLLMValidator(llm_local, search_tool_local)
    validator.set_available_companies(all_companies)
    
    results = []
    
    for idx, row in data.iterrows():
        company_name = row["PARTA2NAMEOFCOMP"]
        operator_id = row["OPERATOR_ID"]
        address = f"{row['PARTA4STREET']}, {row['PARTA4CITY']}, {row['PARTA4STATE']}"
        
        progress_pct = (idx + 1) / len(data) * 100
        print(f"\n{'='*60}")
        print(f"🏢 [{idx+1}/{len(data)} - {progress_pct:.1f}%] {company_name}")
        print(f"{'='*60}")
        
        try:
            # Use agent to find parent
            result = validator.validate_direct(
                company_name=company_name,
                operator_id=operator_id,
                address=address
            )
            
            # Convert JV partners list to comma-separated string
            jv_partners = result.get("jv_partners")
            jv_partners_str = ", ".join(jv_partners) if jv_partners else None
            
            # Get primary operator (uppercase if present)
            primary_operator = result.get("primary_operator")
            if primary_operator:
                primary_operator = primary_operator.upper()
            
            results.append({
                "OPERATOR_ID": operator_id,
                "ORIGINAL_NAME": company_name,
                "IMMEDIATE_PARENT": result["parent"],
                "LLM_SEARCH_PARENT": result.get("llm_search_parent", result["parent"]),
                "IS_JV": result.get("is_jv", False),
                "JV_PARTNERS": jv_partners_str,
                "PRIMARY_OPERATOR": primary_operator,
                "CONFIDENCE": result["confidence"],
                "REASONING": result["reasoning"],
                "ACQUISITION_DATE": result.get("acquisition_date"),
                "RECENT_CHANGE": result.get("recent_change", False),
                "PARTA4STREET": row["PARTA4STREET"],
                "PARTA4CITY": row["PARTA4CITY"],
                "PARTA4STATE": row["PARTA4STATE"],
            })
            
            jv_flag = " [JV]" if result.get("is_jv") else ""
            operator_flag = f" (→{primary_operator})" if primary_operator else ""
            print(f"✅ {result['parent']}{jv_flag}{operator_flag} | Conf: {result['confidence']}")
            
        except Exception as e:
            print(f"❌ Error: {str(e)[:100]}")
            results.append({
                "OPERATOR_ID": operator_id,
                "ORIGINAL_NAME": company_name,
                "IMMEDIATE_PARENT": "ERROR",
                "LLM_SEARCH_PARENT": "ERROR",
                "IS_JV": False,
                "JV_PARTNERS": None,
                "PRIMARY_OPERATOR": None,
                "CONFIDENCE": 0,
                "REASONING": f"Error: {str(e)}",
                "ACQUISITION_DATE": None,
                "RECENT_CHANGE": False,
                "PARTA4STREET": row["PARTA4STREET"],
                "PARTA4CITY": row["PARTA4CITY"],
                "PARTA4STATE": row["PARTA4STATE"],
            })
    
    batch_duration = (datetime.now() - batch_start).total_seconds()
    avg_time = batch_duration / batch_size if batch_size > 0 else 0
    print(f"\n{'#'*60}")
    print(f"✅ BATCH DONE: {batch_size} companies in {batch_duration:.0f}s ({avg_time:.1f}s each)")
    print(f"{'#'*60}\n")
    
    return pd.DataFrame(results)

print("✓ Agent-based UDF defined")

In [ ]:
# Cell 4: Run Agent-Based Analysis

from datetime import datetime

# Get total count
total_companies = companies_df.count()

print("="*60)
print("Starting agent-based parent company analysis...")
print(f"📊 Total companies to process: {total_companies}")
print("🤖 Agent will automatically search and reason for each company")
print("\nThis will take approximately 2-4 hours.")
print("Progress shown per batch (batches run in parallel on serverless)")
print("="*60)

start_time = datetime.now()

# Apply agent-based validation
parent_mappings_df = companies_df.select(
    find_parent_agent(
        struct(
            "OPERATOR_ID",
            "PARTA2NAMEOFCOMP",
            "PARTA4STREET",
            "PARTA4CITY",
            "PARTA4STATE"
        )
    ).alias("result")
).select("result.*")

# Cache and trigger computation
parent_mappings_df.cache()
processed_count = parent_mappings_df.count()

# Final stats
total_time = (datetime.now() - start_time).total_seconds()

print(f"\n{'='*60}")
print(f"✅ PROCESSING COMPLETE!")
print(f"📊 Total processed: {processed_count} companies")
print(f"⏰ Total time: {total_time/60:.1f} minutes ({total_time/3600:.2f} hours)")
print(f"⚡ Average: {processed_count/(total_time/60):.1f} companies/min")
print(f"{'='*60}\n")

# Show sample results
display(parent_mappings_df.limit(20))

In [ ]:
# Cell 5: Build Hierarchy Graphs

parent_mappings_pd = parent_mappings_df.toPandas()
normalizer = CompanyNameNormalizer()

# 1. PHMSA-validated hierarchy (constrained to PHMSA dataset)
parent_mappings_phmsa = parent_mappings_pd.rename(columns={"ORIGINAL_NAME": "child", "IMMEDIATE_PARENT": "parent"})
hierarchy_phmsa_df = graph_builder.build(parent_mappings_phmsa)
hierarchy_phmsa_df = hierarchy_phmsa_df.rename(columns={
    "ultimate_parent": "ultimate_parent_phmsa",
    "hierarchy_path": "hierarchy_path_phmsa",
    "hierarchy_depth": "hierarchy_depth_phmsa"
})

# 2. LLM-discovered hierarchy (includes external parents, normalized names)
parent_mappings_llm = parent_mappings_pd.copy()
parent_mappings_llm['LLM_SEARCH_PARENT_NORMALIZED'] = parent_mappings_llm['LLM_SEARCH_PARENT'].apply(normalizer.normalize)
parent_mappings_llm_norm = parent_mappings_llm.rename(columns={"ORIGINAL_NAME": "child", "LLM_SEARCH_PARENT_NORMALIZED": "parent"})
hierarchy_llm_df = graph_builder.build(parent_mappings_llm_norm)
hierarchy_llm_df = hierarchy_llm_df.rename(columns={
    "ultimate_parent": "ultimate_parent_llm",
    "hierarchy_path": "hierarchy_path_llm",
    "hierarchy_depth": "hierarchy_depth_llm"
})

# Merge both hierarchies
hierarchy_combined_df = hierarchy_phmsa_df.merge(
    hierarchy_llm_df[['company', 'ultimate_parent_llm', 'hierarchy_path_llm', 'hierarchy_depth_llm']],
    on='company', how='outer'
)

print(f"✓ PHMSA hierarchy: {len(hierarchy_phmsa_df)} companies")
print(f"✓ LLM hierarchy: {len(hierarchy_llm_df)} companies")
print(f"✓ Combined: {len(hierarchy_combined_df)} companies")

In [ ]:
# Cell 6: Analysis & Statistics

from pyspark.sql.functions import col, count, avg

# Quick stats
conf_stats = parent_mappings_df.select(avg("CONFIDENCE"), count("*")).collect()[0]
recent_changes = parent_mappings_df.filter(col("RECENT_CHANGE") == True).count()
jv_count = parent_mappings_df.filter(col("IS_JV") == True).count()
external_parents = parent_mappings_df.filter(
    (col("LLM_SEARCH_PARENT") != col("IMMEDIATE_PARENT")) & (col("LLM_SEARCH_PARENT") != "ULTIMATE")
).count()

print(f"Total Companies: {conf_stats[1]}")
print(f"Avg Confidence: {conf_stats[0]:.1f}")
print(f"Joint Ventures: {jv_count}")
print(f"Recent Changes (2024-26): {recent_changes}")
print(f"External Parents Found: {external_parents}")

# Top parent companies
print("\nTop 10 Parent Companies:")
display(parent_mappings_df.groupBy("IMMEDIATE_PARENT").count().orderBy(col("count").desc()).limit(10))

# Recent ownership changes
if recent_changes > 0:
    print("\nRecent Ownership Changes:")
    display(parent_mappings_df.filter(col("RECENT_CHANGE") == True)
            .select("ORIGINAL_NAME", "IMMEDIATE_PARENT", "ACQUISITION_DATE").limit(10))

In [ ]:
# Cell 7: Save Results (Optional)

# Uncomment to save results to Unity Catalog
# output_table = "your_catalog.your_schema.operator_hierarchy_llm"
# 
# hierarchy_spark_df = spark.createDataFrame(hierarchy_combined_df)
# final_df = hierarchy_spark_df.join(
#     parent_mappings_df,
#     hierarchy_spark_df.company == parent_mappings_df.ORIGINAL_NAME,
#     "left"
# )
# 
# final_df.write.mode("overwrite").saveAsTable(output_table)
# print(f"✓ Saved to {output_table}")

print("✓ Analysis complete. Uncomment code above to save results.")